# Utah EITC Reform Analysis (2025)

This notebook analyzes the impact of making Utah's Earned Income Tax Credit (EITC) refundable for families with young children.

## Baseline (Current Law)
- Utah EITC matches 20% of the federal EITC
- The credit is non-refundable

## Reform
- Make the Utah EITC fully refundable for households with children age 2 or younger

## Metrics
We calculate:
- Budgetary impact (net cost)
- Winners (percentage of population affected)
- Overall poverty impact
- Child poverty impact

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_us.reforms.states.ut.ut_refundable_eitc import ut_refundable_eitc
import pandas as pd
import numpy as np

UT_DATASET = "hf://policyengine/policyengine-us-data/states/UT.h5"

## Helper Functions

In [2]:
def calculate_poverty(sim, period=2025, child_only=False):
    """
    Calculate poverty rate and count.
    
    Args:
        sim: Microsimulation object
        period: Year to analyze
        child_only: If True, only count children under 18
    
    Returns:
        poverty_rate: Weighted poverty rate
        people_in_poverty: Weighted count
    """
    age = np.array(sim.calculate("age", period=period))
    is_in_poverty = np.array(sim.calculate("person_in_poverty", period=period))
    person_weight = np.array(sim.calculate("person_weight", period=period))
    
    if child_only:
        mask = age < 18
    else:
        mask = np.ones_like(age, dtype=bool)
    
    # Weighted poverty rate
    weighted_in_poverty = (is_in_poverty[mask] * person_weight[mask]).sum()
    weighted_total = person_weight[mask].sum()
    poverty_rate = weighted_in_poverty / weighted_total if weighted_total > 0 else 0
    
    # Weighted count of people in poverty
    people_in_poverty = weighted_in_poverty
    
    return {
        "poverty_rate": poverty_rate,
        "people_in_poverty": people_in_poverty,
        "total_people": weighted_total
    }

def calculate_budgetary_impact(baseline_sim, reform_sim, variable, period=2025):
    """
    Calculate the budgetary impact (net cost) of a reform.
    """
    baseline_value = baseline_sim.calculate(variable, period=period, map_to="household").sum()
    reform_value = reform_sim.calculate(variable, period=period, map_to="household").sum()
    
    return reform_value - baseline_value

def calculate_winners(baseline_sim, reform_sim, period=2025):
    """
    Calculate winners from a reform at the person level (weighted).
    Winners: People in households with higher net income under reform.
    Returns weighted count and percentage of total population.
    """
    # Get household-level income change
    baseline_income = np.array(baseline_sim.calculate("household_net_income", period=period, map_to="household"))
    reform_income = np.array(reform_sim.calculate("household_net_income", period=period, map_to="household"))
    household_weight = np.array(baseline_sim.calculate("household_weight", period=period))
    income_change = reform_income - baseline_income
    
    # Get person-level data
    household_id_person = np.array(baseline_sim.calculate("household_id", period=period, map_to="person"))
    household_id_household = np.array(baseline_sim.calculate("household_id", period=period, map_to="household"))
    person_weight = np.array(baseline_sim.calculate("person_weight", period=period))
    
    # Create mapping of household_id to income_change
    income_change_dict = dict(zip(household_id_household, income_change))
    
    # Map income change to each person
    person_income_change = np.array([income_change_dict.get(hh_id, 0) for hh_id in household_id_person])
    
    # Weighted count of people who are winners (gained more than $1)
    winners_mask = person_income_change > 1
    people_winning = person_weight[winners_mask].sum()
    total_people = person_weight.sum()
    
    # Calculate percentage
    pct_winners = (people_winning / total_people * 100) if total_people > 0 else 0
    
    # Average gain for winning households (weighted)
    winning_hh_mask = income_change > 1
    if winning_hh_mask.sum() > 0:
        avg_gain = np.average(income_change[winning_hh_mask], weights=household_weight[winning_hh_mask])
    else:
        avg_gain = 0
    
    return {
        "people_winning": people_winning,
        "total_people": total_people,
        "pct_winners": pct_winners,
        "avg_gain": avg_gain
    }

def format_currency(value):
    """Format value as currency in millions."""
    return f"${value/1e6:.2f}M"

def format_percent(value):
    """Format value as percentage."""
    return f"{value*100:.2f}%"

## Define Baseline and Reform

In [3]:
def create_reform():
    """Reform: Make UT EITC refundable for households with children age 2 or younger"""
    param_reform = Reform.from_dict(
        {
            "gov.contrib.states.ut.eitc.in_effect": {
                "2025-01-01.2100-12-31": True
            }
        },
        country_id="us",
    )
    # Combine the parameter reform with the structural reform
    return (param_reform, ut_refundable_eitc)

print("Reform functions defined!")

Reform functions defined!


## Load Simulations

In [4]:
print("Loading baseline (current law - non-refundable UT EITC at 20%)...")
baseline = Microsimulation(dataset=UT_DATASET)
print("Baseline loaded")

print("\nLoading reform (refundable UT EITC for families with young children)...")
reform = create_reform()
reform_sim = Microsimulation(dataset=UT_DATASET, reform=reform)
print("Reform loaded")

print("\n" + "="*60)
print("All simulations ready!")
print("="*60)

Loading baseline (current law - non-refundable UT EITC at 20%)...
Baseline loaded

Loading reform (refundable UT EITC for families with young children)...
Reform loaded

All simulations ready!


## Calculate Impacts

In [5]:
# Baseline metrics
baseline_overall_pov = calculate_poverty(baseline, child_only=False)
baseline_child_pov = calculate_poverty(baseline, child_only=True)

# Reform metrics
reform_overall_pov = calculate_poverty(reform_sim, child_only=False)
reform_child_pov = calculate_poverty(reform_sim, child_only=True)

# Budgetary impact - calculated as increase in household net income (cost to state)
# The refundable EITC variable only exists in reform, so we use net income change
baseline_hh_income = baseline.calculate("household_net_income", period=2025, map_to="household").sum()
reform_hh_income = reform_sim.calculate("household_net_income", period=2025, map_to="household").sum()
eitc_cost = reform_hh_income - baseline_hh_income

# Winners (at person level)
winners = calculate_winners(baseline, reform_sim)

print("All impacts calculated")

All impacts calculated


## Results Summary

In [6]:
print("\n" + "="*80)
print("UTAH EITC REFORM IMPACTS (2025)")
print("Baseline: Non-refundable UT EITC at 20% | Reform: Refundable for families with children age 0-2")
print("="*80)

print(f"\n{'BUDGETARY IMPACT':=^80}")
print(f"UT Refundable EITC net cost:   {format_currency(eitc_cost)}")

print(f"\n{'WINNERS (POPULATION)':=^80}")
print(f"People gaining income:         {winners['people_winning']:,.0f} ({winners['pct_winners']:.2f}% of population)")
print(f"Average gain per household:    ${winners['avg_gain']:,.2f}")

print(f"\n{'POVERTY IMPACT - OVERALL':=^80}")
print(f"Baseline poverty rate:         {format_percent(baseline_overall_pov['poverty_rate'])}")
print(f"Reform poverty rate:           {format_percent(reform_overall_pov['poverty_rate'])}")
overall_pov_reduction = baseline_overall_pov['poverty_rate'] - reform_overall_pov['poverty_rate']
overall_pov_pct_reduction = (overall_pov_reduction / baseline_overall_pov['poverty_rate'] * 100) if baseline_overall_pov['poverty_rate'] > 0 else 0
print(f"Absolute reduction:            {format_percent(overall_pov_reduction)}")
print(f"Relative reduction:            {overall_pov_pct_reduction:.2f}%")
people_lifted = baseline_overall_pov['people_in_poverty'] - reform_overall_pov['people_in_poverty']
print(f"People lifted from poverty:    {people_lifted:,.0f}")

print(f"\n{'POVERTY IMPACT - CHILDREN':=^80}")
print(f"Baseline child poverty rate:   {format_percent(baseline_child_pov['poverty_rate'])}")
print(f"Reform child poverty rate:     {format_percent(reform_child_pov['poverty_rate'])}")
child_pov_reduction = baseline_child_pov['poverty_rate'] - reform_child_pov['poverty_rate']
child_pov_pct_reduction = (child_pov_reduction / baseline_child_pov['poverty_rate'] * 100) if baseline_child_pov['poverty_rate'] > 0 else 0
print(f"Absolute reduction:            {format_percent(child_pov_reduction)}")
print(f"Relative reduction:            {child_pov_pct_reduction:.2f}%")
children_lifted = baseline_child_pov['people_in_poverty'] - reform_child_pov['people_in_poverty']
print(f"Children lifted from poverty:  {children_lifted:,.0f}")
print("="*80)


UTAH EITC REFORM IMPACTS (2025)
Baseline: Non-refundable UT EITC at 20% | Reform: Refundable for families with children age 0-2

================================BUDGETARY IMPACT================================
UT Refundable EITC net cost:   $14.22M

==============================WINNERS (POPULATION)==============================
People gaining income:         98,777 (2.85% of population)
Average gain per household:    $668.61

============================POVERTY IMPACT - OVERALL============================
Baseline poverty rate:         8.32%
Reform poverty rate:           8.04%
Absolute reduction:            0.27%
Relative reduction:            3.29%
People lifted from poverty:    9,497

===========================POVERTY IMPACT - CHILDREN============================
Baseline child poverty rate:   6.37%
Reform child poverty rate:     5.87%
Absolute reduction:            0.50%
Relative reduction:            7.84%
Children lifted from poverty:  4,749


In [7]:
# Calculate households benefitting (weighted)
baseline_hh_income = np.array(baseline.calculate("household_net_income", period=2025, map_to="household"))
reform_hh_income = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="household"))
household_weight = np.array(baseline.calculate("household_weight", period=2025))

hh_income_change = reform_hh_income - baseline_hh_income
hh_benefitting_mask = hh_income_change > 1  # Gained more than $1

households_benefitting = household_weight[hh_benefitting_mask].sum()
total_households = household_weight.sum()
pct_households_benefitting = (households_benefitting / total_households) * 100

print("="*70)
print("HOUSEHOLDS BENEFITTING FROM UT REFUNDABLE EITC")
print("="*70)
print(f"Households benefitting:        {households_benefitting:,.0f}")
print(f"Total households:              {total_households:,.0f}")
print(f"Percentage of households:      {pct_households_benefitting:.2f}%")
print("="*70)

HOUSEHOLDS BENEFITTING FROM UT REFUNDABLE EITC
Households benefitting:        21,262
Total households:              1,015,265
Percentage of households:      2.09%


## Export Results

In [ ]:
# Calculate poverty changes
overall_pov_reduction = baseline_overall_pov['poverty_rate'] - reform_overall_pov['poverty_rate']
overall_pov_pct_reduction = (overall_pov_reduction / baseline_overall_pov['poverty_rate'] * 100) if baseline_overall_pov['poverty_rate'] > 0 else 0
child_pov_reduction = baseline_child_pov['poverty_rate'] - reform_child_pov['poverty_rate']
child_pov_pct_reduction = (child_pov_reduction / baseline_child_pov['poverty_rate'] * 100) if baseline_child_pov['poverty_rate'] > 0 else 0

# Create results DataFrame (reform only)
results = [
    {
        "Scenario": "Reform",
        "Description": "Refundable EITC for children age 0-2",
        "Net Cost": format_currency(eitc_cost),
        "Overall Poverty Change (%)": f"{overall_pov_pct_reduction:.2f}%",
        "Child Poverty Change (%)": f"{child_pov_pct_reduction:.2f}%",
        "% Population Winning": f"{winners['pct_winners']:.2f}%"
    }
]

df_results = pd.DataFrame(results)

print("\n" + "="*110)
print("UT EITC REFORM SUMMARY")
print("="*110)
print(df_results.to_string(index=False))
print("="*110)

# Export to CSV
df_results.to_csv("ut_eitc_reform_results.csv", index=False)
print("\nExported to: ut_eitc_reform_results.csv")


UT EITC REFORM SUMMARY
Scenario                          Description Net Cost Overall Poverty Change (%) Child Poverty Change (%) % Population Winning
  Reform Refundable EITC for children age 0-2  $14.22M                      3.29%                    7.84%                2.85%

Exported to: ut_eitc_reform_results.csv


: 